In [1]:
# Setup - copy-pasted from 2A

# Python ≥3.9 is required
import sys
assert sys.version_info >= (3, 9)

# Scikit-Learn ≥1.0 is required
import sklearn
assert sklearn.__version__ >= "1.0"

# Common imports
import numpy as np
import pandas as pd

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Precision options
np.set_printoptions(precision=2)
pd.options.display.float_format = '{:.3f}'.format

# Statistical analysis and testing
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Ignore useless warnings (see SciPy issue #5998 and seaborn/matplotlib bug)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings(action="ignore", message="The figure layout has changed to tight")

import json
import datetime

In [2]:
#f=open("default-cards-20240806090619.json")
#raw_data = json.load(f)
# raw_data[0]

In [3]:
todays_date = datetime.datetime(year=2025,month=12,day=14)

Features the program gets NOTHING out of:
    "object", 
    "oracle_id", 
    "arena_id", 
    "multiverse_ids", 
    "tcgplayer_id", 
    "cardmarket_id", 
    "uri", 
    "scryfall_uri", 
    "oracle_text",
    "image_status",
    "image_uris",
    "set_name",
    "set_uri",
    "set_search_uri",
    "scryfall_set_uri",
    "rulings_uri",
    "prints_search_uri",
    "preview",
    "related_uris",
    "purchase_uris",
    "flavor_text",
    "usd",
    "usd_foil",
    "usd_etched",
    "eur",
    "eur_foil",
    "highres_image"
\+ maybe some more

Ordinal features I want to give the program:
Rarity

Categorical features I want to give the program:
Colors





There are some features that ML won't make head or tail of (eg. various URIs, oracle_text)

Some features that I deliberately don't want to give to the program (eg. card prices in paper)

Some features I'm interested in from a data analysis perspective, but will very likely be useless for determining price (eg. )

In [4]:
#df = pd.DataFrame.from_dict(raw_data)

Legalities:

Some formats are simple binaries. These are:
- vintage - legal or restricted
- legacy - legal or banned
- penny - legal or not_legal
- commander - legal or banned

(((((Some formats are *effectively* binaries. These are:
- duel - legal or banned (ignore restricted, it's just "is this creature legendary))))))

The rest are more complicated. 

(not_legal, legal, banned) formats:
- future
- standard
- pioneer
- modern
- pauper

(not_legal, legal, restricted) formats:


These need to have two variables:
Binary:
- printed_into_{format} -> = not({format}="not_legal")

Then "ternary" - 0, 1 or NaN (use a mean imputer when doing ML)
- banned_in_{format} -> (banned/restricted)=1, legal=0, not_legal=NaN

4 categories:
paupercommander -> (not_legal, legal, restricted, banned)
Just ignore restricted (it's just the same as "is an uncommon creature")

I think I need to take a step back and ignore legality for a bit 

# Description of features
### Describe every feature in detail

DATA CLEANING

Decide to only care about MTGO prices etc

"object" is never anything except "card" - useless
Remove all Un-set cards I think

Target variable: "prices" -> "tix" 
(MTGO economy uses tix as a currency - 1 tix = 1 US dollar)
Remove all prices except for tix ()

Variables to keep:
MTGO ID 
(Name? I want it for readability but it's useless for the algorithm)


Data cleaning:
Remove anything where "mtgo" not in "games" (ie. Alchemy cards)
Remove Un-set cards (I don't wanna deal with CMC 0.5 or 1,000,000)
Potentially remove P9 as outliers?




Problems for data representation: 
Unpack legalities and prices
(Possibly don't give it legality in "penny" though - dead giveaway!)
Keywords: One hot encoding again but that's gonna get hella annoying - consider just teaching it the most common ones?
Prices: Just keep "tix" (if model is very very shit, consider letting it have other prices lol)

Categorical things to one-hot encode, in order of probable utility to the algorithm:
Format legality - Legal, Not Legal, Banned, Restricted (maybe remove "penny" if I'm feeling mean, but start by not doing that)
Rarity - common, uncommon, rare, mythic, special, bonus (note: bonus is literally just the power 9!)
(Don't bother teaching it colours I think)
Keywords


Problems to solve for the algorithm:

I'd like it to be able to understand keywords - ie. if a card has flying, I want the algorithm to take that into account. ATM keywords are stored in a list - hard for ML to understand!


In [5]:
#csv_cards = pd.DataFrame(json.load(f))
#csv_cards

# Order of operations:
# 1: expand important dict features to columns
# 2: remove outlier rows
# 3: remove junk columns
# 4. onehot-encode things I want (eg. keywords)

In [6]:
import scryfall_preprocessing as sf

In [7]:
from importlib import reload
reload(sf)

<module 'scryfall_preprocessing' from '/Users/charlie/DataScience/mtg_price_analysis/scryfall_preprocessing.py'>

In [8]:
df = sf.get_cards()
df

,object,id,oracle_id,multiverse_ids,mtgo_id,tcgplayer_id,cardmarket_id,name,lang,released_at,...,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,content_warning,flavor_name,printed_type_line,defense,printed_text
0,card,a471b306-4941-4e46-a0cb-d92895c16f8a,00037840-6089-42ec-8c5c-281f9f474504,[692174],137223.000,615195.000,807933.000,"Nissa, Worldsoul Speaker",en,2025-02-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,6b7baaa7-67f2-4902-a4c1-761d58cce1d4,000492bf-7eaa-4939-a51c-4eef74e4c1d1,[],NaN,NaN,NaN,Mine Security,en,2025-08-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,0004ebd0-dfd6-4276-b4a6-de0003e94237,[15862],15870.000,3094.000,3081.000,Static Orb,en,2001-04-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,7050735c-b232-47a6-a342-01795bfd0d46,0006faf6-7a61-426c-9034-579f2cfcfa83,[370780],49283.000,69965.000,262945.000,Sensory Deprivation,en,2013-07-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,9c5e9b5f-6e71-4afc-abbd-ba4c788a3240,00078ea3-0462-4a6e-b7b1-25fea012b2b7,[],NaN,NaN,NaN,Kavaron Consumed,en,2025-08-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36200,card,d6695fa8-881c-407c-91d9-3ac770372d35,fffa9334-3576-4f70-9605-2ad062cdbc69,[430770],64646.000,136683.000,298834.000,Without Weakness,en,2017-07-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36201,card,4d5edf4f-5695-42fc-9e57-c4faef60fbc3,fffaa634-42a0-4038-b027-24f28754fec7,[571548],101914.000,276869.000,665173.000,Firesong and Sunspeaker,en,2022-07-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36202,card,22a6a5f1-1405-4efb-af3e-e1f58d664e99,fffcb71f-e802-436e-917d-eaa6607ab74f,"[503766, 503767]",87643.000,230214.000,530502.000,"Toralf, God of Fury // Toralf's Hammer",en,2021-02-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36203,card,09f06f55-7918-46c4-80ff-0bf39e091a4a,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,[430833],64772.000,132234.000,298282.000,"Samut, the Tested",en,2017-07-14,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All variables I want to one-hot encode:

- Types
- Keywords
- Set code
- Legalities (one for each format GUHHHH)
- Border color


UPDATE: Things I actually care about:
- "power/cmc", "toughness/cmc", "(power+toughness)/cmc" variables?
- RELEVANT keywords only (just the common ones)
- NUMBER of colours
- NUMBER of coloured pips

(note: might be worth using a separate df for just creatures for some of this?)

All variables needing imputation:

- Power
- Toughness
- Mana cost (for transform cards lol)

In [9]:
import scryfall_preprocessing as sf
#import get_scryfall_data

df2 = sf.expand_dict_features(df)
df3 = sf.standard_only(df2)
df4 = sf.drop_junk_features(df3)
#df5 = sf.drop_more_features(df4)
df5 = sf.remove_legalities(df4)
df6 = sf.ordinate_rarities(df5)

In [10]:
df6

,name,released_at,layout,mana_cost,cmc,type_line,power,toughness,colors,color_identity,...,security_stamp,booster,edhrec_rank,penny_rank,resource_id,flavor_name,printed_type_line,printed_text,eur,ordinate_rarity
0,"Greta, Sweettooth Scourge",2023-09-08,normal,{1}{B}{G},3.000,Legendary Creature — Human Warrior,3,3,"[B, G]","[B, G]",...,NaN,True,4730.000,3821.000,NaN,NaN,NaN,NaN,0.13,2
1,Seedship Agrarian,2025-08-01,normal,{3}{G},4.000,Creature — Insect Scientist,3,3,[G],[G],...,NaN,True,12986.000,NaN,2D06A4B98579996F08D6116C41925C4D0680F2FC0B4299...,NaN,NaN,NaN,0.12,2
2,Escape Tunnel,2025-08-01,normal,,0.000,Land,NaN,NaN,[],[],...,NaN,False,1018.000,368.000,841BB455438516DD32C817041EFEB14DF8F2937319ED13...,NaN,NaN,NaN,0.22,1
3,Food Fight,2023-09-08,normal,{1}{R},2.000,Enchantment,NaN,NaN,[R],[R],...,oval,True,18013.000,4998.000,NaN,NaN,NaN,NaN,0.08,3
4,Witch's Mark,2023-09-08,normal,{1}{R},2.000,Sorcery,NaN,NaN,[R],[R],...,NaN,True,3634.000,3705.000,NaN,NaN,NaN,NaN,0.14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,Twinmaw Stormbrood // Charring Bite,2025-04-11,adventure,{5}{W} // {1}{R},6.000,Creature — Dragon // Sorcery — Omen,5,4,[W],"[R, W]",...,NaN,True,15034.000,1422.000,NaN,NaN,NaN,NaN,0.11,2
3685,Wildheart Invoker,2024-11-15,normal,{2}{G}{G},4.000,Creature — Elf Shaman,4,3,[G],[G],...,NaN,False,12334.000,13675.000,NaN,NaN,NaN,NaN,0.09,1
3686,Auxiliary Boosters,2025-08-01,normal,{4}{W},5.000,Artifact — Equipment,NaN,NaN,[W],[W],...,NaN,True,19623.000,NaN,D08AE5129952ACC16A999111F183AB945377034198FC3D...,NaN,NaN,NaN,0.04,1
3687,Withering Torment,2024-09-27,normal,{2}{B},3.000,Instant,NaN,NaN,[B],[B],...,NaN,True,381.000,3447.000,NaN,NaN,NaN,NaN,2.11,2


In [11]:
reload(sf)

<module 'scryfall_preprocessing' from '/Users/charlie/DataScience/mtg_price_analysis/scryfall_preprocessing.py'>

In [12]:
df7 = sf.convert_all_to_numbers(df6)

In [13]:
df7

,name,released_at,layout,mana_cost,type_line,colors,color_identity,keywords,game_changer,variation,...,flavor_name,printed_type_line,printed_text,ordinate_rarity,foil,booster,eur,cmc,power,toughness
0,"Greta, Sweettooth Scourge",2023-09-08,normal,{1}{B}{G},Legendary Creature — Human Warrior,"[B, G]","[B, G]",[Food],False,False,...,NaN,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000
1,Seedship Agrarian,2025-08-01,normal,{3}{G},Creature — Insect Scientist,[G],[G],[Landfall],False,False,...,NaN,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000
2,Escape Tunnel,2025-08-01,normal,,Land,[],[],[],False,False,...,NaN,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000
3,Food Fight,2023-09-08,normal,{1}{R},Enchantment,[R],[R],[],False,False,...,NaN,NaN,NaN,3,1,1,1.000,1.000,0.000,0.000
4,Witch's Mark,2023-09-08,normal,{1}{R},Sorcery,[R],[R],[Role token],False,False,...,NaN,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,Twinmaw Stormbrood // Charring Bite,2025-04-11,adventure,{5}{W} // {1}{R},Creature — Dragon // Sorcery — Omen,[W],"[R, W]",[Flying],False,False,...,NaN,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000
3685,Wildheart Invoker,2024-11-15,normal,{2}{G}{G},Creature — Elf Shaman,[G],[G],[],False,False,...,NaN,NaN,NaN,1,1,1,1.000,1.000,1.000,1.000
3686,Auxiliary Boosters,2025-08-01,normal,{4}{W},Artifact — Equipment,[W],[W],[Equip],False,False,...,NaN,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000
3687,Withering Torment,2024-09-27,normal,{2}{B},Instant,[B],[B],[],False,False,...,NaN,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000


In [14]:
df8 = sf.cut_subtypes(df7)

In [15]:
df8

,name,released_at,layout,mana_cost,colors,color_identity,keywords,game_changer,variation,set,...,printed_type_line,printed_text,ordinate_rarity,foil,booster,eur,cmc,power,toughness,types
0,"Greta, Sweettooth Scourge",2023-09-08,normal,{1}{B}{G},"[B, G]","[B, G]",[Food],False,False,woe,...,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000,"[Legendary, Creature]"
1,Seedship Agrarian,2025-08-01,normal,{3}{G},[G],[G],[Landfall],False,False,eoe,...,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000,[Creature]
2,Escape Tunnel,2025-08-01,normal,,[],[],[],False,False,eoc,...,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000,[Land]
3,Food Fight,2023-09-08,normal,{1}{R},[R],[R],[],False,False,woe,...,NaN,NaN,3,1,1,1.000,1.000,0.000,0.000,[Enchantment]
4,Witch's Mark,2023-09-08,normal,{1}{R},[R],[R],[Role token],False,False,woe,...,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000,[Sorcery]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,Twinmaw Stormbrood // Charring Bite,2025-04-11,adventure,{5}{W} // {1}{R},[W],"[R, W]",[Flying],False,False,tdm,...,NaN,NaN,2,1,1,1.000,1.000,1.000,1.000,"[Creature, —, Dragon, //, Sorcery]"
3685,Wildheart Invoker,2024-11-15,normal,{2}{G}{G},[G],[G],[],False,False,fdn,...,NaN,NaN,1,1,1,1.000,1.000,1.000,1.000,[Creature]
3686,Auxiliary Boosters,2025-08-01,normal,{4}{W},[W],[W],[Equip],False,False,eoe,...,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000,[Artifact]
3687,Withering Torment,2024-09-27,normal,{2}{B},[B],[B],[],False,False,dsk,...,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000,[Instant]


In [16]:
df8.dtypes

name                  object
released_at           object
layout                object
mana_cost             object
colors                object
color_identity        object
keywords              object
game_changer            bool
variation               bool
set                   object
set_type              object
watermark             object
security_stamp        object
edhrec_rank          float64
penny_rank           float64
resource_id           object
flavor_name           object
printed_type_line     object
printed_text          object
ordinate_rarity        int64
foil                   int64
booster                int64
eur                  float64
cmc                  float64
power                float64
toughness            float64
types                 object
dtype: object

In [17]:
from sklearn.preprocessing import OneHotEncoder
def expand_typeline(data: pd.DataFrame) -> pd.DataFrame:
    engineered_data = data.copy()
    types = engineered_data.types
    relevant_types = ["Creature","Artifact"]
    print("...unfinished!")
    #return engineered_data

In [18]:
#x = expand_typeline(df8)
#x

In [19]:
df8[np.logical_not(df8.colors.notnull())]

,name,released_at,layout,mana_cost,colors,color_identity,keywords,game_changer,variation,set,...,printed_type_line,printed_text,ordinate_rarity,foil,booster,eur,cmc,power,toughness,types
66,The Emperor of Palamecia // The Lord Master of...,2025-06-13,transform,NaN,NaN,"[R, U]","[Starfall, Transform]",False,False,fin,...,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000,"[Legendary, Creature, —, Human, Noble, Wizard,..."
180,Treasure Map // Treasure Cove,2023-11-17,transform,NaN,NaN,[],"[Transform, Treasure, Scry]",False,False,lci,...,NaN,NaN,3,1,1,1.000,1.000,0.000,0.000,"[Artifact, //, Land]"
188,Sidequest: Card Collection // Magicked Card,2025-06-13,transform,NaN,NaN,[U],"[Flying, Transform, Crew]",False,False,fin,...,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000,"[Enchantment, //, Artifact]"
335,"Matzalantli, the Great Door // The Core",2023-11-17,transform,NaN,NaN,[],"[Transform, Fathomless descent]",False,False,lci,...,NaN,NaN,3,1,1,1.000,1.000,0.000,0.000,"[Legendary, Artifact, //, Legendary, Land]"
402,Kaslem's Stonetree // Kaslem's Strider,2023-11-17,transform,NaN,NaN,[G],"[Craft, Transform]",False,False,lci,...,NaN,NaN,1,1,1,1.000,1.000,0.000,0.000,"[Artifact, //, Artifact, Creature]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3600,Spring-Loaded Sawblades // Bladewheel Chariot,2023-11-17,transform,NaN,NaN,[W],"[Craft, Transform, Crew, Flash]",False,False,lci,...,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000,"[Artifact, //, Artifact]"
3602,Jade Seedstones // Jadeheart Attendant,2023-11-17,transform,NaN,NaN,[G],"[Craft, Transform]",False,False,lci,...,NaN,NaN,2,1,1,1.000,1.000,0.000,0.000,"[Artifact, //, Artifact, Creature]"
3650,Unstable Glyphbridge // Sandswirl Wanderglyph,2023-11-17,transform,NaN,NaN,[W],"[Flying, Craft, Transform]",False,False,lci,...,NaN,NaN,3,1,1,1.000,1.000,0.000,0.000,"[Artifact, //, Artifact, Creature]"
3651,"Aclazotz, Deepest Betrayal // Temple of the Dead",2023-11-17,transform,NaN,NaN,[B],"[Flying, Lifelink, Transform]",False,False,lci,...,NaN,NaN,4,1,1,1.000,1.000,0.000,0.000,"[Legendary, Creature]"


In [20]:
#df8[np.logical_not(df8.edhrec_rank.notnull())]

# Wastes is the only card to not have an "edh_rank"

In [21]:
#df8[np.logical_not(df8.colors.notnull())]  
# 67 of these

#df8[np.logical_or((df8.layout=="modal_dfc"),(df8.layout=="transform"))]   
# also 67 of these!

# They do still have tix/eur and cmc values but don't have various datapoints eg. mana_cost, colors, etc.

In [22]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(categories='auto')

#train_data_cat = train_data[["zipcode_group"]]
#train_data_cat_1hot = cat_encoder.fit_transform(train_data_cat)
#train_data_cat_1hot

In [23]:
# Need to expand the dict features BEFORE using the transformer, because need to drop the "tix" column before transforming

In [24]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):

#     def transform(self, data, labels=None) -> pd.DataFrame:
#         # let's make a copy of the original dataset
#         engineered_data = data.copy()

#         engineered_data = expand_dict_features(engineered_data) # type: ignore
#         engineered_data = drop_junk_features(engineered_data)
#         engineered_data = clean_dataset(engineered_data)
        


#         return engineered_data


In [25]:
#fe_trf = FeatureEngineeringTransformer()
#fe_trf.transform(df)

In [26]:
#df4.to_clipboard()

# Take a sample, then put it on clipboard

In [27]:
# Variables I want!!

# Number of colors

# power/cmc
# toughness/cmc
# p+t/cmc

In [28]:
def num_colors(data: pd.DataFrame) -> pd.DataFrame:
    engineered_data = data.copy()
    num_colors = engineered_data.colors.str.len()
    engineered_data["num_colors"] = num_colors
    engineered_data = engineered_data.drop(columns="colors")
    return engineered_data

In [29]:
def vars_wrt_cmc(data: pd.DataFrame) -> pd.DataFrame:
    engineered_data = data.copy()
    power, toughness, cmc = engineered_data["power"], engineered_data["toughness"], engineered_data["cmc"]

    p_vs_cmc = (power.div(cmc)).rename("power/cmc")
    t_vs_cmc = (toughness.div(cmc)).rename("toughness/cmc")
    pt_vs_cmc = ((power.add(toughness)).div(cmc)).rename("p+t/cmc")

    engineered_data = engineered_data.join(p_vs_cmc).join(t_vs_cmc).join(pt_vs_cmc)
    return engineered_data

In [30]:
df9 = num_colors(df8)

In [31]:
df9.describe()

,edhrec_rank,penny_rank,ordinate_rarity,foil,booster,eur,cmc,power,toughness,num_colors
count,3683.000,1692.000,3689.000,3689.000,3689.000,3689.000,3689.000,3689.000,3689.000,3614.000
mean,11239.456,5917.345,2.075,1.000,1.000,1.000,1.000,0.570,0.570,1.056
std,6475.394,4101.029,0.938,0.000,0.000,0.000,0.000,0.495,0.495,0.601
min,12.000,16.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000
25%,5896.000,2358.500,1.000,1.000,1.000,1.000,1.000,0.000,0.000,1.000
50%,11267.000,5338.000,2.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
75%,16119.000,8837.250,3.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
max,28164.000,17131.000,4.000,1.000,1.000,1.000,1.000,1.000,1.000,5.000


In [32]:
df10 = vars_wrt_cmc(df9)

In [33]:
df10

,name,released_at,layout,mana_cost,color_identity,keywords,game_changer,variation,set,set_type,...,booster,eur,cmc,power,toughness,types,num_colors,power/cmc,toughness/cmc,p+t/cmc
0,"Greta, Sweettooth Scourge",2023-09-08,normal,{1}{B}{G},"[B, G]",[Food],False,False,woe,expansion,...,1,1.000,1.000,1.000,1.000,"[Legendary, Creature]",2.000,1.000,1.000,2.000
1,Seedship Agrarian,2025-08-01,normal,{3}{G},[G],[Landfall],False,False,eoe,expansion,...,1,1.000,1.000,1.000,1.000,[Creature],1.000,1.000,1.000,2.000
2,Escape Tunnel,2025-08-01,normal,,[],[],False,False,eoc,commander,...,1,1.000,1.000,0.000,0.000,[Land],0.000,0.000,0.000,0.000
3,Food Fight,2023-09-08,normal,{1}{R},[R],[],False,False,woe,expansion,...,1,1.000,1.000,0.000,0.000,[Enchantment],1.000,0.000,0.000,0.000
4,Witch's Mark,2023-09-08,normal,{1}{R},[R],[Role token],False,False,woe,expansion,...,1,1.000,1.000,0.000,0.000,[Sorcery],1.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3684,Twinmaw Stormbrood // Charring Bite,2025-04-11,adventure,{5}{W} // {1}{R},"[R, W]",[Flying],False,False,tdm,expansion,...,1,1.000,1.000,1.000,1.000,"[Creature, —, Dragon, //, Sorcery]",1.000,1.000,1.000,2.000
3685,Wildheart Invoker,2024-11-15,normal,{2}{G}{G},[G],[],False,False,fdn,core,...,1,1.000,1.000,1.000,1.000,[Creature],1.000,1.000,1.000,2.000
3686,Auxiliary Boosters,2025-08-01,normal,{4}{W},[W],[Equip],False,False,eoe,expansion,...,1,1.000,1.000,0.000,0.000,[Artifact],1.000,0.000,0.000,0.000
3687,Withering Torment,2024-09-27,normal,{2}{B},[B],[],False,False,dsk,expansion,...,1,1.000,1.000,0.000,0.000,[Instant],1.000,0.000,0.000,0.000
